In [1]:
!vimr 20.txt

In [80]:
from pathlib import Path


data = '''###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############'''
data = Path('20.txt').read_text()

In [81]:
maze = []
for y,l in enumerate(data.splitlines()):
    for x,c in enumerate(l):
        if c != '#':
            
            maze.append((x,y))
        if c == 'S':
            start = (x, y)
        if c == 'E':
            end = (x, y)
        

In [82]:
start, end , maze

((61, 49),
 (69, 69),
 [(1, 1),
  (2, 1),
  (3, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1),
  (92, 1),
  (93, 1),
  (94, 1),
  (95, 1),
  (97, 1),
  (98, 1),
  (99, 1),
  (101, 1),
  (102, 1),
  (103, 1),
  (105, 1),
  (106, 1),
  (107, 1),
  (108, 1),

In [83]:
nsew = [(-1,0),(1,0),(0, -1),(0, 1)]

In [84]:
def demijik(maze, start):
    dist = dict()
    open = [(0, start)]
    while open:
        score, (x, y) = open.pop()
        dist[(x,y)] = score
        if (x,y) == end:
            return dist
        neigh = [(x+dx, y+dy) for (dx, dy) in nsew if (x+dx, y+dy) in maze and  (x+dx, y+dy) not in dist]
        assert len(neigh) == 1, (neigh, dist)
        nx, ny = neigh[0]
        open.append((score+1, (nx, ny)))
        

race = demijik(maze, start)

In [85]:
race

{(61, 49): 0,
 (61, 48): 1,
 (61, 47): 2,
 (62, 47): 3,
 (63, 47): 4,
 (64, 47): 5,
 (65, 47): 6,
 (66, 47): 7,
 (67, 47): 8,
 (67, 48): 9,
 (67, 49): 10,
 (68, 49): 11,
 (69, 49): 12,
 (70, 49): 13,
 (71, 49): 14,
 (71, 48): 15,
 (71, 47): 16,
 (71, 46): 17,
 (71, 45): 18,
 (70, 45): 19,
 (69, 45): 20,
 (69, 44): 21,
 (69, 43): 22,
 (70, 43): 23,
 (71, 43): 24,
 (72, 43): 25,
 (73, 43): 26,
 (73, 42): 27,
 (73, 41): 28,
 (72, 41): 29,
 (71, 41): 30,
 (71, 40): 31,
 (71, 39): 32,
 (71, 38): 33,
 (71, 37): 34,
 (72, 37): 35,
 (73, 37): 36,
 (73, 38): 37,
 (73, 39): 38,
 (74, 39): 39,
 (75, 39): 40,
 (75, 38): 41,
 (75, 37): 42,
 (75, 36): 43,
 (75, 35): 44,
 (75, 34): 45,
 (75, 33): 46,
 (75, 32): 47,
 (75, 31): 48,
 (75, 30): 49,
 (75, 29): 50,
 (75, 28): 51,
 (75, 27): 52,
 (75, 26): 53,
 (75, 25): 54,
 (75, 24): 55,
 (75, 23): 56,
 (74, 23): 57,
 (73, 23): 58,
 (73, 24): 59,
 (73, 25): 60,
 (73, 26): 61,
 (73, 27): 62,
 (73, 28): 63,
 (73, 29): 64,
 (73, 30): 65,
 (73, 31): 66,
 (73,

In [86]:
max(race.values()) 

9452

In [87]:
def count(race):
    for (x,y),t in race.items():
        pot_short = [(x+2*dx, y+2*dy) for (dx, dy) in nsew]
        t_short = [race.get((x,y), None) for (x,y) in pot_short]
        yield from [nt-t-2 for nt  in t_short if nt is not None]
    
    

In [88]:
from collections import Counter

```
There are 14 cheats that save 2 picoseconds.
There are 14 cheats that save 4 picoseconds.
There are 2 cheats that save 6 picoseconds.
There are 4 cheats that save 8 picoseconds.
There are 2 cheats that save 10 picoseconds.
There are 3 cheats that save 12 picoseconds.
There is one cheat that saves 20 picoseconds.
There is one cheat that saves 36 picoseconds.
There is one cheat that saves 38 picoseconds.
There is one cheat that saves 40 picoseconds.
There is one cheat that saves 64 picoseconds.
```

In [89]:
ct = Counter([c for c in count(race) if c>0]) ## 4108 to hight

In [90]:
for k in sorted(ct.keys()):
    print(ct[k], ' that saves', k ,'picos')

980  that saves 2 picos
980  that saves 4 picos
263  that saves 6 picos
433  that saves 8 picos
194  that saves 10 picos
329  that saves 12 picos
138  that saves 14 picos
227  that saves 16 picos
89  that saves 18 picos
178  that saves 20 picos
76  that saves 22 picos
162  that saves 24 picos
67  that saves 26 picos
123  that saves 28 picos
52  that saves 30 picos
88  that saves 32 picos
39  that saves 34 picos
81  that saves 36 picos
36  that saves 38 picos
72  that saves 40 picos
29  that saves 42 picos
63  that saves 44 picos
26  that saves 46 picos
63  that saves 48 picos
32  that saves 50 picos
59  that saves 52 picos
26  that saves 54 picos
47  that saves 56 picos
19  that saves 58 picos
46  that saves 60 picos
19  that saves 62 picos
39  that saves 64 picos
14  that saves 66 picos
37  that saves 68 picos
17  that saves 70 picos
30  that saves 72 picos
13  that saves 74 picos
30  that saves 76 picos
9  that saves 78 picos
25  that saves 80 picos
12  that saves 82 picos
28  that s

In [95]:
sum([v for k,v in ct.items() if k >=100]) # 1494 too low

1524